In [1]:
import torch
import sys
import k_diffusion as K
import torch._C._onnx as _C_onnx
from torchinfo import summary
import cv2
import numpy as np
import torchvision

import pynvml
import onnxruntime as ort
import os
import torch.nn as nn
from PIL import Image

/home/ubuntu/anaconda3/envs/k-diff-dist/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_float32_matmul_precision('high')

In [3]:
config = "/home/ubuntu/transformer-distillation/configs/hdit-shifted-windows/head1.json"
config = K.config.load_config(config)

model = K.config.make_model(config).cuda()

# dct = torch.load("200_net_G_hdit.pth")#.keys()
# dct = {
#     key.replace('model.', ''): value for key, value in dct.items()
# }

# model.load_state_dict(dct)
final_activation_function = nn.Tanh()
print(sum([p.numel() for p in model.parameters()]) / 1e6)
model.eval()

img = torch.from_numpy(np.array(Image.open("input_0_24_62.png")))
img = img / 255.
img = img * 2. - 1.
img = img.transpose(-1, -2).transpose(-2, -3).unsqueeze(0).cuda()

cst = torch.ones((img.shape[0])).cuda() * 500
with torch.no_grad(): out = final_activation_function(model(img, cst)[0])

out = out / 2. + .5
out = out.squeeze(0).transpose(0, 1).transpose(1, 2)
out = Image.fromarray((out.data.cpu().numpy() * 255.).astype(np.uint8))
out.save('out_shifted.png')

85.079396


ValueError: torch.Size([1, 1024, 1024, 3])

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self, model, device):
        super(MyModel, self).__init__()
        # Example: simple linear layer
        self.cst = torch.ones((1)).to(device) *500
        self.model = model

    def forward(self, img):
        x  = self.model(img,self.cst)
        return x

In [ ]:
model.eval()

pmodel = MyModel(model, 'cpu')
of = 'ps1.onnx'
input = torch.tensor(cv2.imread('input_0_24_62.png')).permute(2,0,1).unsqueeze(0).cuda()/255.
torch.onnx.export(pmodel.cpu(), (input.cpu()), of, opset_version = 17,do_constant_folding=True, export_params=True,input_names = ['input'],output_names = ['output'], verbose=False)

/home/ubuntu/transformer-distillation/k-diffusion-onnx/k_diffusion/models/axial_rope.py:54: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  y_min = torch.max(torch.tensor(-1), -1 / ar_adj)
/home/ubuntu/transformer-distillation/k-diffusion-onnx/k_diffusion/models/axial_rope.py:55: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  y_max = torch.min(torch.tensor(1), 1 / ar_adj)
/home/ubuntu/transformer-distillation/k-diffusion-onnx/k_diffusion/models/axial_rope.py:57: TracerWarning: tor

In [ ]:
input.shape

torch.Size([1, 3, 1024, 1024])

In [ ]:
providers = [
    ('CUDAExecutionProvider')]

sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
sess_options.enable_mem_pattern = False
sess_options.use_deterministic_compute = True
sess_options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
sess_options.enable_cpu_mem_arena = False





session = ort.InferenceSession("ps1.onnx",sess_options,providers=providers)

# Run inference with ONNX Runtime


input_data = input.cpu().numpy().astype(np.float32)
ort_inputs = {session.get_inputs()[0].name: input_data}
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
for _ in range(5):
    start.record()
    onnx_output = session.run(None, ort_inputs)
    end.record()
    print(start.elapsed_time(end))

# Compare the outputs


/home/ubuntu/anaconda3/envs/OMGD/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


4258.6513671875
